In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ELT_PySpark") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.2,com.amazonaws:aws-java-sdk-bundle:1.11.901") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

25/05/23 13:24:20 WARN Utils: Your hostname, lenovo-PC resolves to a loopback address: 127.0.1.1; using 172.31.112.46 instead (on interface eth0)
25/05/23 13:24:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntucoy/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntucoy/.ivy2/cache
The jars for the packages stored in: /home/ubuntucoy/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-12d173b4-07db-46b6-9681-0aeb63867fa6;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 1536ms :: artifacts dl 31ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.2 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.11.1026] in [default]
	-----------------------------------------------------

In [2]:
df = spark.read.csv("s3a://raw/crm/prd_info.csv", header=True, inferSchema=True)
df.show()

25/05/23 13:27:25 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

+------+----------------+--------------------+--------+--------+------------+----------+
|prd_id|         prd_key|              prd_nm|prd_cost|prd_line|prd_start_dt|prd_end_dt|
+------+----------------+--------------------+--------+--------+------------+----------+
|   210|CO-RF-FR-R92B-58|HL Road Frame - B...|    NULL|      R |  2003-07-01|      NULL|
|   211|CO-RF-FR-R92R-58|HL Road Frame - R...|    NULL|      R |  2003-07-01|      NULL|
|   212| AC-HE-HL-U509-R|Sport-100 Helmet-...|      12|      S |  2011-07-01|2007-12-28|
|   213| AC-HE-HL-U509-R|Sport-100 Helmet-...|      14|      S |  2012-07-01|2008-12-27|
|   214| AC-HE-HL-U509-R|Sport-100 Helmet-...|      13|      S |  2013-07-01|      NULL|
|   215|   AC-HE-HL-U509|Sport-100 Helmet-...|      12|      S |  2011-07-01|2007-12-28|
|   216|   AC-HE-HL-U509|Sport-100 Helmet-...|      14|      S |  2012-07-01|2008-12-27|
|   217|   AC-HE-HL-U509|Sport-100 Helmet-...|      13|      S |  2013-07-01|      NULL|
|   218| CL-SO-SO-B90

In [3]:
from pyspark.sql.functions import ( 
    col, trim, upper, when, substring, coalesce, expr, lead, regexp_replace, length
)
from pyspark.sql.window import Window

# Window spec untuk LEAD
window_spec = Window.partitionBy("prd_key").orderBy("prd_start_dt")

# Transformasi
df_transformed = (
    df.select(
        "prd_id",
        regexp_replace(substring("prd_key", 1, 5), "-", "_").alias("cat_id"),   # Extract category id
        expr("substring(prd_key, 7,length(prd_key))").alias("prd_key"), # Extract product key
        "prd_nm",
        coalesce(col("prd_cost"), expr("0")).alias("prd_cost"),   # Null handling cost
        when(upper(trim(col("prd_line"))) == "M", "Mountain")
            .when(upper(trim(col("prd_line"))) == "R", "Road")
            .when(upper(trim(col("prd_line"))) == "S", "Other Sales")
            .when(upper(trim(col("prd_line"))) == "T", "Touring")
            .otherwise("n/a")
            .alias("prd_line"),
        # Convert ke DATE (anggap sudah bertipe timestamp/string yang bisa dikonversi otomatis)
        col("prd_start_dt").cast("date").alias("prd_start_dt"),
        (lead("prd_start_dt").over(window_spec).cast("date") - expr("INTERVAL 1 day")).alias("prd_end_dt") # LEAD window untuk menghitung end date
    )
    .orderBy("prd_id") 
)
df_transformed.show()

+------+------+----------+--------------------+--------+-----------+------------+----------+
|prd_id|cat_id|   prd_key|              prd_nm|prd_cost|   prd_line|prd_start_dt|prd_end_dt|
+------+------+----------+--------------------+--------+-----------+------------+----------+
|   210| CO_RF|FR-R92B-58|HL Road Frame - B...|       0|       Road|  2003-07-01|      NULL|
|   211| CO_RF|FR-R92R-58|HL Road Frame - R...|       0|       Road|  2003-07-01|      NULL|
|   212| AC_HE| HL-U509-R|Sport-100 Helmet-...|      12|Other Sales|  2011-07-01|2012-06-30|
|   213| AC_HE| HL-U509-R|Sport-100 Helmet-...|      14|Other Sales|  2012-07-01|2013-06-30|
|   214| AC_HE| HL-U509-R|Sport-100 Helmet-...|      13|Other Sales|  2013-07-01|      NULL|
|   215| AC_HE|   HL-U509|Sport-100 Helmet-...|      12|Other Sales|  2011-07-01|2012-06-30|
|   216| AC_HE|   HL-U509|Sport-100 Helmet-...|      14|Other Sales|  2012-07-01|2013-06-30|
|   217| AC_HE|   HL-U509|Sport-100 Helmet-...|      13|Other Sales|  

In [4]:
df_transformed.write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("s3a://clean/crm/prd_info_clean.csv")

25/05/23 13:30:32 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/05/23 13:30:34 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
                                                                                

In [5]:
df_cleaned = spark.read.option("header", True).csv("s3a://clean/crm/prd_info_clean.csv")
df_cleaned.show()

+------+------+----------+--------------------+--------+-----------+------------+----------+
|prd_id|cat_id|   prd_key|              prd_nm|prd_cost|   prd_line|prd_start_dt|prd_end_dt|
+------+------+----------+--------------------+--------+-----------+------------+----------+
|   210| CO_RF|FR-R92B-58|HL Road Frame - B...|       0|       Road|  2003-07-01|      NULL|
|   211| CO_RF|FR-R92R-58|HL Road Frame - R...|       0|       Road|  2003-07-01|      NULL|
|   212| AC_HE| HL-U509-R|Sport-100 Helmet-...|      12|Other Sales|  2011-07-01|2012-06-30|
|   213| AC_HE| HL-U509-R|Sport-100 Helmet-...|      14|Other Sales|  2012-07-01|2013-06-30|
|   214| AC_HE| HL-U509-R|Sport-100 Helmet-...|      13|Other Sales|  2013-07-01|      NULL|
|   215| AC_HE|   HL-U509|Sport-100 Helmet-...|      12|Other Sales|  2011-07-01|2012-06-30|
|   216| AC_HE|   HL-U509|Sport-100 Helmet-...|      14|Other Sales|  2012-07-01|2013-06-30|
|   217| AC_HE|   HL-U509|Sport-100 Helmet-...|      13|Other Sales|  

In [6]:
import duckdb

# Convert dari Spark ke Pandas
df_cleaned_prd_info = df_cleaned.toPandas()

# Connect ke DuckDB
con = duckdb.connect("/mnt/d/data_engineering/duckdb/db/dev.duckdb")

# Simpan sebagai tabel DuckDB (view)
con.register("crm_prd_info_clean_view", df_cleaned_prd_info)

# Simpan sebagai tabel DuckDB (bukan view)
con.execute("CREATE OR REPLACE TABLE crm_prd_info_clean AS SELECT * FROM crm_prd_info_clean_view")

In [7]:
con.execute("SELECT * FROM crm_prd_info_clean LIMIT 10").df().style.hide(axis="index")

prd_id,cat_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,prd_end_dt
210,CO_RF,FR-R92B-58,HL Road Frame - Black- 58,0,Road,2003-07-01,None
211,CO_RF,FR-R92R-58,HL Road Frame - Red- 58,0,Road,2003-07-01,None
212,AC_HE,HL-U509-R,Sport-100 Helmet- Red,12,Other Sales,2011-07-01,2012-06-30
213,AC_HE,HL-U509-R,Sport-100 Helmet- Red,14,Other Sales,2012-07-01,2013-06-30
214,AC_HE,HL-U509-R,Sport-100 Helmet- Red,13,Other Sales,2013-07-01,None
215,AC_HE,HL-U509,Sport-100 Helmet- Black,12,Other Sales,2011-07-01,2012-06-30
216,AC_HE,HL-U509,Sport-100 Helmet- Black,14,Other Sales,2012-07-01,2013-06-30
217,AC_HE,HL-U509,Sport-100 Helmet- Black,13,Other Sales,2013-07-01,None
218,CL_SO,SO-B909-M,Mountain Bike Socks- M,3,Mountain,2011-07-01,None
219,CL_SO,SO-B909-L,Mountain Bike Socks- L,3,Mountain,2011-07-01,None


In [8]:
con.close()